<a href="https://colab.research.google.com/github/Weinihsiang/Pytorch/blob/main/Lab4_Data_Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Data Imputation using an Autoencoder


In this lab, you will build and train an autoencoder to impute (or "fill in") missing data.

We will be using the
Adult Data Set provided by the UCI Machine Learning Repository [1], available
at https://archive.ics.uci.edu/ml/datasets/adult.
The data set contains census record files of adults, including their
age, martial status, the type of work they do, and other features.

Normally, people use this data set to build a supervised classification
model to classify whether a person is a high income earner.
We will not use the dataset for this original intended purpose.

Instead, we will perform the task of imputing (or "filling in") missing values in the dataset. For example,
we may be missing one person's martial status, and another person's age, and
a third person's level of education. Our model will predict the missing features
based on the information that we do have about each person.

We will use a variation of a denoising autoencoder to solve this data imputation
problem. Our autoencoder will be trained using inputs that have one categorical feature artificially
removed, and the goal of the autoencoder is to correctly reconstruct all features,
including the one removed from the input.

In the process, you are expected to learn to:

1. Clean and process continuous and categorical data for machine learning.
2. Implement an autoencoder that takes continuous and categorical (one-hot) inputs.
3. Tune the hyperparameters of an autoencoder.
4. Use baseline models to help interpret model performance.

[1] Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.


### What to submit

Submit a PDF file containing all your code, outputs, and write-up. You can produce a PDF of your Google Colab file by going to File > Print and then save as PDF. The Colab instructions have more information.

Do not submit any other files produced by your code.

Include a link to your colab file in your submission.


## Colab Link

Include a link to your Colab file here. If you would like the TA to look at your
Colab file in case your solutions are cut off, **please make sure that your Colab
file is publicly accessible at the time of submission**.

Colab Link: https://colab.research.google.com/drive/1jmPqmlq3m0I8Sk63ZzrQCTeYzdQfv-Nf?usp=sharing


In [ ]:
import csv
import numpy as np
import random
import torch
import torch.utils.data

#normalize the features
from sklearn.preprocessing import MinMaxScaler

import time
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt # for plotting
import torch.optim as optim #for gradient descent

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import shutil
from torchvision.datasets import ImageFolder


## Part 0

We will be using a package called `pandas` for this assignment.

If you are using Colab, `pandas` should already be available.
If you are using your own computer,
installation instructions for `pandas` are available here:
https://pandas.pydata.org/pandas-docs/stable/install.html

In [ ]:
import pandas as pd

# Part 1. Data Cleaning [15 pt]

The adult.data file is available at `https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data`

The function `pd.read_csv` loads the adult.data file into a pandas dataframe.
You can read about the pandas documentation for `pd.read_csv` at
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
header = ['age', 'work', 'fnlwgt', 'edu', 'yredu', 'marriage', 'occupation',
 'relationship', 'race', 'sex', 'capgain', 'caploss', 'workhr', 'country']
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=header,
    index_col=False)

<ipython-input-3-037957db2593>:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


In [ ]:
df.shape # there are 32561 rows (records) in the data frame, and 14 columns (features)

(32561, 14)

### Part (a) Continuous Features [3 pt]

For each of the columns `["age", "yredu", "capgain", "caploss", "workhr"]`, report the minimum, maximum, and average value across the dataset.

Then, normalize each of the features `["age", "yredu", "capgain", "caploss", "workhr"]`
so that their values are always between 0 and 1.
Make sure that you are actually modifying the dataframe `df`.

Like numpy arrays and torch tensors,
pandas data frames can be sliced. For example, we can
display the first 3 rows of the data frame (3 records) below.

In [ ]:
df[:3] # show the first 3 records

age               work  fnlwgt         edu  yredu             marriage  \
0   39          State-gov   77516   Bachelors     13        Never-married   
1   50   Self-emp-not-inc   83311   Bachelors     13   Married-civ-spouse   
2   38            Private  215646     HS-grad      9             Divorced   

           occupation    relationship    race    sex  capgain  caploss  \
0        Adm-clerical   Not-in-family   White   Male     2174        0   
1     Exec-managerial         Husband   White   Male        0        0   
2   Handlers-cleaners   Not-in-family   White   Male        0        0   

   workhr         country  
0      40   United-States  
1      13   United-States  
2      40   United-States

Alternatively, we can slice based on column names,
for example `df["race"]`, `df["hr"]`, or even index multiple columns
like below.

In [ ]:
subdf = df[["age", "yredu", "capgain", "caploss", "workhr"]]
subdf[:3] # show the first 3 records

age  yredu  capgain  caploss  workhr
0   39     13     2174        0      40
1   50     13        0        0      13
2   38      9        0        0      40

Numpy works nicely with pandas, like below:

In [ ]:
np.sum(subdf["caploss"])

2842700

Just like numpy arrays, you can modify
entire columns of data rather than one scalar element at a time.
For example, the code  

`df["age"] = df["age"] + 1`

would increment everyone's age by 1.

In [ ]:
#find mininum and maximum

columns_of_interest = ["age", "yredu", "capgain", "caploss", "workhr"]
statistics = df[columns_of_interest].describe().loc[["min", "max", "mean"]]

print(statistics)


            age      yredu       capgain     caploss     workhr
min   17.000000   1.000000      0.000000     0.00000   1.000000
max   90.000000  16.000000  99999.000000  4356.00000  99.000000
mean  38.581647  10.080679   1077.648844    87.30383  40.437456


In [ ]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns to be normalized
columns_to_normalize = ["age", "yredu", "capgain", "caploss", "workhr"]

# Normalize the selected columns in the DataFrame
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])


In [ ]:
df[:3] # show the first 3 records

age               work  fnlwgt         edu     yredu  \
0  0.301370          State-gov   77516   Bachelors  0.800000   
1  0.452055   Self-emp-not-inc   83311   Bachelors  0.800000   
2  0.287671            Private  215646     HS-grad  0.533333   

              marriage          occupation    relationship    race    sex  \
0        Never-married        Adm-clerical   Not-in-family   White   Male   
1   Married-civ-spouse     Exec-managerial         Husband   White   Male   
2             Divorced   Handlers-cleaners   Not-in-family   White   Male   

   capgain  caploss    workhr         country  
0  0.02174      0.0  0.397959   United-States  
1  0.00000      0.0  0.122449   United-States  
2  0.00000      0.0  0.397959   United-States

### Part (b) Categorical Features [1 pt]

What percentage of people in our data set are male? Note that the data labels all have an unfortunate space in the beginning, e.g. " Male" instead of "Male".

What percentage of people in our data set are female?

In [ ]:
# hint: you can do something like this in pandas
print("Number of Male:", sum(df["sex"] == " Male"),
      "Number of Female", sum(df["sex"] == " Female"),"Total Num: ", df["sex"].shape[0])

male_percentage = round((sum(df["sex"] == " Male") / df["sex"].shape[0]) * 100, 2)
female_percentage = round((sum(df["sex"] == " Female") / df["sex"].shape[0]) * 100, 2)

print(male_percentage, "% of people in our data set are male.")
print(female_percentage, "% of people in our data set are female.")


Number of Male: 21790 Number of Female 10771 Total Num:  32561
66.92 % of people in our data set are male.
33.08 % of people in our data set are female.


### Part (c) [2 pt]

Before proceeding, we will modify our data frame in a couple more ways:

1. We will restrict ourselves to using a subset of the features (to simplify our autoencoder)
2. We will remove any records (rows) already containing missing values, and store them in a second dataframe. We will only use records without missing values to train our autoencoder.

Both of these steps are done for you, below.

How many records contained missing features? What percentage of records were removed?

In [ ]:
contcols = ["age", "yredu", "capgain", "caploss", "workhr"]
catcols = ["work", "marriage", "occupation", "edu", "relationship", "sex"]
features = contcols + catcols
df = df[features]

In [ ]:
missing = pd.concat([df[c] == " ?" for c in catcols], axis=1).any(axis=1)
df_with_missing = df[missing]
df_not_missing = df[~missing]

In [ ]:
#check the size of missing and without missing

df_not_missing.shape, df_with_missing.shape, df.shape

((30718, 11), (1843, 11), (32561, 11))

In [ ]:
percentage_removed = (df_with_missing.shape[0] / df.shape[0]) * 100

print(f"Number of records with missing features: {df_with_missing.shape[0]}")
print(f"Percentage of records removed: {percentage_removed:.2f}%")

Number of records with missing features: 1843
Percentage of records removed: 5.66%


In [ ]:
df_not_missing[:3] # show the first 3 records

age     yredu  capgain  caploss    workhr               work  \
0  0.301370  0.800000  0.02174      0.0  0.397959          State-gov   
1  0.452055  0.800000  0.00000      0.0  0.122449   Self-emp-not-inc   
2  0.287671  0.533333  0.00000      0.0  0.397959            Private   

              marriage          occupation         edu    relationship    sex  
0        Never-married        Adm-clerical   Bachelors   Not-in-family   Male  
1   Married-civ-spouse     Exec-managerial   Bachelors         Husband   Male  
2             Divorced   Handlers-cleaners     HS-grad   Not-in-family   Male

### Part (d) One-Hot Encoding [1 pt]

What are all the possible values of the feature "work" in `df_not_missing`? You may find the Python function `set` useful.

In [ ]:
work_set = set(df_not_missing["work"])

print("Possible values of the 'work' feature:")
for value in work_set:
    print(value)

Possible values of the 'work' feature:
 Local-gov
 Without-pay
 Self-emp-inc
 Self-emp-not-inc
 State-gov
 Private
 Federal-gov


We will be using a one-hot encoding to represent each of the categorical variables.
Our autoencoder will be trained using these one-hot encodings.

We will use the pandas function `get_dummies` to produce one-hot encodings
for all of the categorical variables in `df_not_missing`.

In [ ]:
data = pd.get_dummies(df_not_missing)

In [ ]:
data[:3]

age     yredu  capgain  caploss    workhr  work_ Federal-gov  \
0  0.301370  0.800000  0.02174      0.0  0.397959                  0   
1  0.452055  0.800000  0.00000      0.0  0.122449                  0   
2  0.287671  0.533333  0.00000      0.0  0.397959                  0   

   work_ Local-gov  work_ Private  work_ Self-emp-inc  work_ Self-emp-not-inc  \
0                0              0                   0                       0   
1                0              0                   0                       1   
2                0              1                   0                       0   

   ...  edu_ Prof-school  edu_ Some-college  relationship_ Husband  \
0  ...                 0                  0                      0   
1  ...                 0                  0                      1   
2  ...                 0                  0                      0   

   relationship_ Not-in-family  relationship_ Other-relative  \
0                            1                             0   
1                            0                             0   
2                            1                             0   

   relationship_ Own-child  relationship_ Unmarried  relationship_ Wife  \
0                        0                        0                   0   
1                        0                        0                   0   
2                        0                        0                   0   

   sex_ Female  sex_ Male  
0            0          1  
1            0          1  
2            0          1  

[3 rows x 57 columns]

### Part (e) One-Hot Encoding [2 pt]

The dataframe `data` contains the cleaned and normalized data that we will use to train our denoising autoencoder.

How many **columns** (features) are in the dataframe `data`?

Briefly explain where that number come from.

# Answer:
The number of columns of **57** in the DataFrame data include both original numerical or continuous features as well as the additional columns created through one-hot encoding for categorical variables.

In [ ]:
print(f"{data.shape[1]} columns are in the dataframe data")

work_type = len(set(df_not_missing["work"]))
marriage_type = len(set(df_not_missing["marriage"]))
occupation_type = len(set(df_not_missing["occupation"]))
edu_type = len(set(df_not_missing["edu"]))
relationship_type = len(set(df_not_missing["relationship"]))
sex_type = len(set(df_not_missing["sex"]))

total_num = work_type + marriage_type + occupation_type + edu_type + relationship_type + sex_type

print("The number come from one-hot encoding: ", total_num + 5 )

57 columns are in the dataframe data
The number come from one-hot encoding:  57


### Part (f) One-Hot Conversion [3 pt]

We will convert the pandas data frame `data` into numpy, so that
it can be further converted into a PyTorch tensor.
However, in doing so, we lose the column label information that
a panda data frame automatically stores.

Complete the function `get_categorical_value` that will return
the named value of a feature given a one-hot embedding.
You may find the global variables `cat_index` and `cat_values`
useful. (Display them and figure out what they are first.)

We will need this function in the next part of the lab
to interpret our autoencoder outputs. So, the input
to our function `get_categorical_values` might not
actually be "one-hot" -- the input may instead
contain real-valued predictions from our neural network.

In [ ]:
datanp = data.values.astype(np.float32)

In [ ]:
cat_index = {}  # Mapping of feature -> start index of feature in a record
cat_values = {} # Mapping of feature -> list of categorical values the feature can take

# build up the cat_index and cat_values dictionary
for i, header in enumerate(data.keys()):
    if "_" in header: # categorical header
        feature, value = header.split()
        feature = feature[:-1] # remove the last char; it is always an underscore
        if feature not in cat_index:
            cat_index[feature] = i
            cat_values[feature] = [value]
        else:
            cat_values[feature].append(value)
#print(cat_values)
#print(cat_index)

def get_onehot(record, feature):
    """
    Return the portion of `record` that is the one-hot encoding
    of `feature`. For example, since the feature "work" is stored
    in the indices [5:12] in each record, calling `get_range(record, "work")`
    is equivalent to accessing `record[5:12]`.

    Args:
        - record: a numpy array representing one record, formatted
                  the same way as a row in `data.np`
        - feature: a string, should be an element of `catcols`
    """
    start_index = cat_index[feature]
    stop_index = cat_index[feature] + len(cat_values[feature])
    return record[start_index:stop_index]

def get_categorical_value(onehot, feature):
    """
    Return the categorical value name of a feature given
    a one-hot vector representing the feature.

    Args:
        - onehot: a numpy array one-hot representation of the feature
        - feature: a string, should be an element of `catcols`

    Examples:

    >>> get_categorical_value(np.array([0., 0., 0., 0., 0., 1., 0.]), "work")
    'State-gov'
    >>> get_categorical_value(np.array([0.1, 0., 1.1, 0.2, 0., 1., 0.]), "work")
    'Private'
    """
    # <----- TODO: WRITE YOUR CODE HERE ----->
    # You may find the variables `cat_index` and `cat_values`
    # (created above) useful.

    max_index = np.argmax(onehot)

    return cat_values[feature][max_index]

print(get_categorical_value(np.array([0., 0., 0., 0., 0., 1., 0.]), "work"))
print(get_categorical_value(np.array([0.1, 0., 1.1, 0.2, 0., 1., 0.]), "work"))

State-gov
Private


In [ ]:
# more useful code, used during training, that depends on the function
# you write above

def get_feature(record, feature):
    """
    Return the categorical feature value of a record
    """
    onehot = get_onehot(record, feature)
    print(onehot)
    return get_categorical_value(onehot, feature)

def get_features(record):
    """
    Return a dictionary of all categorical feature values of a record
    """
    return { f: get_feature(record, f) for f in catcols }

### Part (g) Train/Test Split [3 pt]

Randomly split the data into approximately 70% training, 15% validation and 15% test.

Report the number of items in your training, validation, and test set.

In [ ]:
# set the numpy seed for reproducibility
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.seed.html
np.random.seed(50)

#shuffle first
np.random.shuffle(datanp)

train_index = int(datanp.shape[0] * 0.7)
val_index = int(datanp.shape[0] * 0.85)

train_data = datanp[:train_index]
val_data = datanp[train_index:val_index]
test_data = datanp[val_index:]

# todo
print("Train data: ", train_data.shape[0],",", round(train_data.shape[0]/30718*100, 3),"percent")
print("Val data: ", val_data.shape[0],",", round(val_data.shape[0]/30718*100,3),"percent")
print("Test data: ", test_data.shape[0],",", round(test_data.shape[0]/30718*100,3), "percent")


Train data:  21502 , 69.998 percent
Val data:  4608 , 15.001 percent
Test data:  4608 , 15.001 percent


## Part 2. Model Setup [5 pt]

### Part (a) [4 pt]

Design a fully-connected autoencoder by modifying the `encoder` and `decoder`
below.

The input to this autoencoder will be the features of the `data`, with
one categorical feature recorded as "missing". The output of the autoencoder
should be the reconstruction of the same features, but with the missing
value filled in.

**Note**: Do not reduce the dimensionality of the input too much!
The output of your embedding is expected to contain information
about ~11 features.

In [ ]:
from torch import nn

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        #self.name = "AutoEncoder"
        self.encoder = nn.Sequential(
            nn.Linear(57, 38), # TODO -- FILL OUT THE CODE HERE!
            nn.Linear(38, 25),
            nn.Linear(25, 16)
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 25),
            nn.Linear(25, 38),
            nn.Linear(38, 57), # TODO -- FILL OUT THE CODE HERE!
            nn.Sigmoid() # get to the range (0, 1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### Part (b) [1 pt]

Explain why there is a sigmoid activation in the last step of the decoder.

(**Note**: the values inside the data frame `data` and the training code in Part 3 might be helpful.)

# Answers:


The sigmoid activation function is utilized in the decoder to restrict the reconstructed output within the range of 0 and 1, aligning with the normalized and scaled nature of the input data. By employing sigmoid, the autoencoder ensures that the output values stay within this specific range, making it suitable for tasks such as image reconstruction or data generation that require maintaining the original data's normalization and scale.

## Part 3. Training [18]

### Part (a) [6 pt]

We will train our autoencoder in the following way:

- In each iteration, we will hide one of the categorical features using the `zero_out_random_features` function
- We will pass the data with one missing feature through the autoencoder, and obtain a reconstruction
- We will check how close the reconstruction is compared to the original data -- including the value of the missing feature

Complete the code to train the autoencoder, and plot the training and validation loss every few iterations.
You may also want to plot training and validation "accuracy" every few iterations, as we will define in
part (b). You may also want to checkpoint your model every few iterations or epochs.

Use `nn.MSELoss()` as your loss function. (Side note: you might recognize that this loss function is not
ideal for this problem, but we will use it anyway.)

In [ ]:
def zero_out_feature(records, feature):
    """ Set the feature missing in records, by setting the appropriate
    columns of records to 0
    """
    start_index = cat_index[feature]
    stop_index = cat_index[feature] + len(cat_values[feature])
    records[:, start_index:stop_index] = 0
    return records

def zero_out_random_feature(records):
    """ Set one random feature missing in records, by setting the
    appropriate columns of records to 0
    """
    return zero_out_feature(records, random.choice(catcols))

def train(model, train_loader, valid_loader, batch_size=64, num_epochs=5, learning_rate=1e-4):
    """ Training loop. You should update this."""
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loss = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    train_acc = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)

    start_time = time.time()


    for epoch in range(num_epochs):
        total_loss = 0
        n = 0 # the number of iterations
        for data in train_loader:

            datam = zero_out_random_feature(data.clone()) # zero out one categorical feature
            recon = model(datam)
            loss = criterion(recon, data)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


            # Save the current training information

            total_loss += loss.item()
            n += 1

        #train_acc[epoch] = get_accuracy(model, train_loader)
        #val_acc[epoch] = get_accuracy(model, valid_loader)
        train_loss[epoch] = total_loss/(n + 1)
        val_loss[epoch] = get_loss(model, valid_loader, criterion)


        print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(
                epoch+1,
                train_acc[epoch],
                val_acc[epoch]))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch)
        torch.save(model.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.4f} seconds".format(elapsed_time))


    # plotting
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.title("Training Curve")
    plt.plot(range(1 ,num_epochs+1), train_loss, label="Train")
    plt.plot(range(1 ,num_epochs+1), val_loss, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

    plt.subplot(1,2,2)
    plt.title("Training Curve")
    plt.plot(range(1 ,num_epochs+1), train_acc, label="Train")
    plt.plot(range(1 ,num_epochs+1), val_acc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

    return train_acc, val_acc, train_loss, val_loss, num_epochs


### Part (b) [3 pt]

While plotting training and validation loss is valuable, loss values are harder to compare
than accuracy percentages. It would be nice to have a measure of "accuracy" in this problem.

Since we will only be imputing missing categorical values, we will define an accuracy measure.
For each record and for each categorical feature, we determine whether
the model can predict the categorical feature given all the other features of the record.

A function `get_accuracy` is written for you. It is up to you to figure out how to
use the function. **You don't need to submit anything in this part.**
To earn the marks, correctly plot the training and validation accuracy every few
iterations as part of your training curve.

In [ ]:
#helper function get_model_name
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values
    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

def get_accuracy(model, data_loader):
    """Return the "accuracy" of the autoencoder model across a data set.
    That is, for each record and for each categorical feature,
    we determine whether the model can successfully predict the value
    of the categorical feature given all the other features of the
    record. The returned "accuracy" measure is the percentage of times
    that our model is successful.

    Args:
       - model: the autoencoder model, an instance of nn.Module
       - data_loader: an instance of torch.utils.data.DataLoader

    Example (to illustrate how get_accuracy is intended to be called.
             Depending on your variable naming this code might require
             modification.)

        >>> model = AutoEncoder()
        >>> vdl = torch.utils.data.DataLoader(data_valid, batch_size=256, shuffle=True)
        >>> get_accuracy(model, vdl)
    """
    total = 0
    acc = 0
    for col in catcols:
        for item in data_loader: # minibatches
            inp = item.detach().numpy()
            out = model(zero_out_feature(item.clone(), col)).detach().numpy()
            for i in range(out.shape[0]): # record in minibatch
                acc += int(get_feature(out[i], col) == get_feature(inp[i], col))
                total += 1
    return acc / total

#function for validation loss
def get_loss(model, data_loader, criterion):
  total_loss = 0
  i = 0
  for data in data_loader:
    data_new = zero_out_random_feature(data.clone()) # zero out one categorical feature
    recon = model(data_new)
    loss = criterion(recon, data)
    total_loss += loss.item()
    i += 1

  return total_loss/(i + 1)

### Part (c) [4 pt]

Run your updated training code, using reasonable initial hyperparameters.

Include your training curve in your submission.

In [ ]:
model = AutoEncoder()
vdl = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
get_accuracy(model, vdl)

Streaming output truncated to the last 5000 lines.
[0.49270234 0.44760016]
[0. 1.]
[0.48769933 0.44851527]
[1. 0.]
[0.4851955  0.45383126]
[0. 1.]
[0.49145904 0.44964603]
[0. 1.]
[0.49004123 0.44836223]
[1. 0.]
[0.4852695  0.45574182]
[0. 1.]
[0.4884157 0.4521215]
[0. 1.]
[0.4905285  0.45299938]
[0. 1.]
[0.49152544 0.4456132 ]
[0. 1.]
[0.4874956 0.4496699]
[0. 1.]
[0.48834553 0.45405844]
[1. 0.]
[0.4884307 0.4517997]
[0. 1.]
[0.48974198 0.45153606]
[0. 1.]
[0.48722866 0.452624  ]
[0. 1.]
[0.48791748 0.45214182]
[1. 0.]
[0.4844201  0.45442355]
[0. 1.]
[0.48857853 0.44906896]
[1. 0.]
[0.48704466 0.45305794]
[0. 1.]
[0.488608   0.45388302]
[0. 1.]
[0.49178553 0.44954848]
[1. 0.]
[0.49106035 0.44638392]
[0. 1.]
[0.48553738 0.45444602]
[0. 1.]
[0.48563308 0.4551342 ]
[0. 1.]
[0.48611328 0.45391235]
[0. 1.]
[0.49246943 0.4483651 ]
[0. 1.]
[0.48643914 0.4527409 ]
[0. 1.]
[0.4885553  0.44742697]
[0. 1.]
[0.48969084 0.45111772]
[1. 0.]
[0.48401615 0.45376626]
[0. 1.]
[0.49046686 0.44901544]
[0.

0.18943199082255915

# Answers:


I chose to set the number of epochs to 10, the batch size to 64, and the learning rate to 1e-4 as my reasonable initial hyperparameters for training the autoencoder. These values strike a balance between model convergence and computational efficiency. To visualize the training curve, I plotted the training loss and validation loss at each epoch using the recorded loss values.

In [ ]:
auto_model = AutoEncoder()

batch_size = 64
use_cuda = True
print("auto_model")
auto_model1 = AutoEncoder()
auto_model1.name = "my_auto_model"
print(auto_model1)

#load the data
train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               shuffle=True) # shuffle after every epoch
valid_loader = torch.utils.data.DataLoader(val_data,
                                               batch_size=batch_size,
                                               shuffle=True) # shuffle after every epoch
test_loader = torch.utils.data.DataLoader(test_data,
                                               batch_size=batch_size,
                                               shuffle=True) # shuffle after every epoch
train_acc, val_acc, train_loss, val_loss, epochs = train(
    auto_model1, train_loader, valid_loader,
    batch_size=batch_size, num_epochs=30, learning_rate=1e-4
)

auto_model
AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=57, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=25, bias=True)
    (2): Linear(in_features=25, out_features=16, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=25, bias=True)
    (1): Linear(in_features=25, out_features=40, bias=True)
    (2): Linear(in_features=40, out_features=57, bias=True)
    (3): Sigmoid()
  )
)
Epoch 1: Train acc: 0.0 |Validation acc: 0.0
Epoch 2: Train acc: 0.0 |Validation acc: 0.0
Epoch 3: Train acc: 0.0 |Validation acc: 0.0
Epoch 4: Train acc: 0.0 |Validation acc: 0.0
Epoch 5: Train acc: 0.0 |Validation acc: 0.0
Epoch 6: Train acc: 0.0 |Validation acc: 0.0
Epoch 7: Train acc: 0.0 |Validation acc: 0.0
Epoch 8: Train acc: 0.0 |Validation acc: 0.0
Epoch 9: Train acc: 0.0 |Validation acc: 0.0
Epoch 10: Train acc: 0.0 |Validation acc: 0.0
Epoch 11: Train acc: 0.0 |Validation acc: 0.0
Epoch 12: Train acc: 0.0 |Validation acc

Final Training Accuracy: 0.0
Final Validation Accuracy: 0.0


### Part (d) [5 pt]

Tune your hyperparameters, training at least 4 different models (4 sets of hyperparameters).

Do not include all your training curves. Instead, explain what hyperparameters
you tried, what their effect was, and what your thought process was as you
chose the next set of hyperparameters to try.

# Answers:

**Frist Model:** batch_size=32, num_epochs=20, learning_rate=1e-4

 I chose a batch size of 32, 20 epochs, and a learning rate of 1e-4 to train the model, aiming for a balance between computational efficiency and model convergence over a relatively longer training duration.

**Second Model:** batch_size=64, num_epochs=10, learning_rate=1e-3

I selected a batch size of 64, 10 epochs, and a learning rate of 1e-3 to train the model, aiming for faster convergence with a larger batch size and a higher learning rate.

**Third Model:** batch_size=128, num_epochs=5, learning_rate=1e-3

I opted for a batch size of 128, 5 epochs, and a learning rate of 1e-3 to train the model, focusing on faster training using a larger batch size while limiting the number of epochs due to computational constraints.

**Fourth Model:** batch_size=64, num_epochs=25, learning_rate=1e-5

I used a batch size of 64, 25 epochs, and a lower learning rate of 1e-5 to train the model for a longer duration, aiming for fine-grained optimization with a smaller learning rate to potentially achieve better performance.


## Part 4. Testing [12 pt]

### Part (a) [2 pt]

Compute and report the test accuracy.

### Part (b) [4 pt]

Based on the test accuracy alone, it is difficult to assess whether our model
is actually performing well. We don't know whether a high accuracy is due to
the simplicity of the problem, or if a poor accuracy is a result of the inherent
difficulty of the problem.

It is therefore very important to be able to compare our model to at least one
alternative. In particular, we consider a simple **baseline**
model that is not very computationally expensive. Our neural network
should at least outperform this baseline model. If our network is not much
better than the baseline, then it is not doing well.

For our data imputation problem, consider the following baseline model:
to predict a missing feature, the baseline model will look at the **most common value** of the feature in the training set.

For example, if the feature "marriage" is missing, then this model's prediction will be the most common value for "marriage" in the training set, which happens to be "Married-civ-spouse".

What would be the test accuracy of this baseline model?


### Part (c) [1 pt]

How does your test accuracy from part (a) compared to your basline test accuracy in part (b)?

### Part (d) [1 pt]

Look at the first item in your test data.
Do you think it is reasonable for a human
to be able to guess this person's education level
based on their other features? Explain.

### Part (e) [2 pt]

What is your model's prediction of this person's education
level, given their other features?


### Part (f) [2 pt]

What is the baseline model's prediction
of this person's education level?